In [ ]:
#| code_fold: true
#| export
from abc import abstractmethod, ABC

## Doubly LinkedList

In [ ]:
#| export

class DoublyLinkedListNode:
    def __init__(self, value, next: "DoublyLinkedListNode"=None, prev: "DoublyLinkedList"=None):
        self.value = value
        self.next = next
        self.prev = prev

class DoublyLinkedList:
    def __init__(self):
        self.head = None
        self.tail = None

    def add(self, node: DoublyLinkedListNode):
        if not self.head:
            self.head = self.tail = node
        else:
            self.tail.next = node
            node.prev = self.tail
            self.tail = self.tail.next

    def remove(self, node: DoublyLinkedListNode):
        if self.head==node:
            self.head = self.head.next
        elif self.tail==node:
            self.tail = self.tail.prev
        else:
            node.prev.next, node.next.prev = node.next, node.prev
        node.next = node.prev = None
        return node

    def __str__(self):
        x = []
        t = self.head
        while t:
            x.append(str(t.value))
            t = t.next
        return "<->".join(x)

    def __repr__(self):
        return self.__str__()

## Storage

In [ ]:
#| export

class Storage:
    def __init__(self, capacity):
        self.capacity = capacity
        self.mapp = dict()

    def get(self, key: str):
        if key in self.mapp:
            return self.mapp[key]
        else:
            raise KeyError

    def put(self, key: str, value: str):
        if len(self.mapp)==self.capacity:
            raise OverflowError
        self.mapp[key] = value

    def delete(self, key: str):
        self.mapp.pop(key)

## Eviction policies

In [ ]:
#| export

class EvictionPolicy(ABC):
    @abstractmethod
    def key_accessed(self, key: str): pass

    @abstractmethod
    def remove(self): pass

class LRUPolicy(EvictionPolicy):
    def __init__(self):
        self.value_node_map = dict()
        self.ll = DoublyLinkedList()

    def key_accessed(self, key: str):
        if key not in self.value_node_map:
            node = DoublyLinkedListNode(key)
            self.ll.add(node)
            self.value_node_map[key] = node
        else:
            node = self.value_node_map[key]
            node = self.ll.remove(node)
            self.ll.add(node)

    def remove(self):
        node = self.ll.head
        self.ll.head = node.next
        self.ll.head.prev = None
        self.value_node_map.pop(node.value)
        return node.value


## Cache class

In [ ]:
#| export

class Cache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.storage = Storage(capacity)
        self.eviction_policy = LRUPolicy()

    def get(self, key):
        self.eviction_policy.key_accessed(key)
        return self.storage.get(key)

    def put(self, key, value):
        try:
            self.storage.put(key, value)
        except OverflowError as overflow:
            key_to_remove = self.eviction_policy.remove()
            print(f"{self.capacity=} reached {key_to_remove=}")
            self.storage.delete(key_to_remove)
            self.storage.put(key, value)
        self.eviction_policy.key_accessed(key)
    

In [ ]:
lru_cache = Cache(5)
actions = [
    lru_cache.put('1', 'a'),
    lru_cache.put('2', 'b'),
    lru_cache.get('1'),
    lru_cache.put('3', 'c'),
    lru_cache.put('4', 'd'),
    lru_cache.put('5', 'e'),
    lru_cache.put('6', 'f'),
    lru_cache.get('1'),
    lru_cache.put('7', 'g'),
    lru_cache.get('4'),
    lru_cache.put('8', 'h'),
]

self.capacity=5 reached key_to_remove='2'
self.capacity=5 reached key_to_remove='3'
self.capacity=5 reached key_to_remove='5'
